In [3]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [4]:
import os
path = os.getcwd()
path

'/home/jovyan/PlanetaryComputerExamples/Kaggle IMDB'

In [5]:
os.listdir(path)

In [6]:
#loading training and testing datasets
import pandas as pd
train_data=pd.read_csv(path + '/labeledTrainData.tsv',header = 0, delimiter = '\t')
test_data=pd.read_csv(path + '/testData.tsv',header = 0, delimiter = '\t')

In [8]:
from scipy.sparse import hstack # 水平堆叠稀疏矩阵(按列)
from sklearn.feature_extraction.text import TfidfVectorizer #把原始文本转化为tf-idf的特征矩阵，文本相似度计算

In [9]:
all_text=train_data['review']
train_text=train_data['review']
y=train_data['sentiment']

In [10]:
# 词袋向量
# tf-idf 统计的方法统计词频
word_vectorizer = TfidfVectorizer(sublinear_tf=True, # 将tf替换为1 + log(tf)
                                  strip_accents='unicode', # 在预处理步骤中删除重音符号并执行其他字符规范化
                                  analyzer='word', # 特征是由单词组成
                                  token_pattern=r'\w{1,}', # 等价于 '[^A-Za-z0-9_]'，至少匹配一次
                                  stop_words='english', # 直接过滤停用词
                                  ngram_range=(1, 1), # 仅使用双字符
                                  max_features=10000) # 只考虑根据语料库中的词汇频率排序的顶部
word_vectorizer.fit(all_text) # 从训练集学习词汇和idf
word_vectorizer

TfidfVectorizer(max_features=10000, stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')

In [11]:
train_word_features = word_vectorizer.transform(train_text) # 转换成特征矩阵

In [12]:
train_word_features

<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 2031487 stored elements in Compressed Sparse Row format>

In [14]:
char_vectorizer = TfidfVectorizer(sublinear_tf=True, 
                                  strip_accents='unicode', 
                                  analyzer='char',
                                  stop_words='english', 
                                  ngram_range=(2, 6), 
                                  max_features=50000)

In [15]:
char_vectorizer.fit(all_text)
char_vectorizer

TfidfVectorizer(analyzer='char', max_features=50000, ngram_range=(2, 6),
                stop_words='english', strip_accents='unicode',
                sublinear_tf=True)

In [16]:
train_char_features = char_vectorizer.transform(train_text)
train_char_features

<25000x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 76764116 stored elements in Compressed Sparse Row format>

In [17]:
train_features = hstack([train_char_features, train_word_features]) # 矩阵拼接，特征是词和字符
train_features

<25000x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 78795603 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, y,test_size=0.2,random_state=42)

from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

lr.fit(X_train,y_train)
preds=lr.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
print("classification_report")
print(classification_report(preds,y_test))

classification_report
              precision    recall  f1-score   support

           0       0.90      0.91      0.90      2459
           1       0.91      0.90      0.90      2541

    accuracy                           0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000



In [25]:
print("confusion_matrix")
confusion_matrix(preds,y_test)

confusion_matrix


array([[2228,  231],
       [ 253, 2288]])

In [20]:
test_text = test_data['review']
all_ = pd.concat([train_text, test_text])
word_vectorizer.fit(all_)
word_vectorizer

TfidfVectorizer(max_features=10000, stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')

In [21]:
train_word_features_ = word_vectorizer.transform(train_text)
test_word_features_ = word_vectorizer.transform(test_text)

In [26]:
char_vectorizer.fit(all_)

TfidfVectorizer(analyzer='char', max_features=50000, ngram_range=(2, 6),
                stop_words='english', strip_accents='unicode',
                sublinear_tf=True)

In [27]:
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [30]:
train_features_ = hstack([train_char_features, train_word_features_])
test_features_ = hstack([test_char_features, test_word_features_])

In [32]:
lr=LogisticRegression(C=0.1,solver='sag') # 随机梯度下降, lambda = 0.1
lr.fit(train_features,y)
final_preds=lr.predict(test_features_)

In [33]:
test_data['sentiment']=final_preds
test_data=test_data[['id','sentiment']]
test_data.to_csv(path + '/sampleSubmission.csv',index=False)

In [ ]:
preds=clf.predict(X_test)
clf.score(X_test, preds)